In [0]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 3.6MB/s 


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import json, os
import pandas as pd
import tqdm
import re
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from string import digits
punct = punctuation+'«»—…“”*№–'+digits
from itertools import combinations
import numpy as np
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
mystem_analyzer = Mystem()
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian") 

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [0]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
pd.set_option('display.max_colwidth', 1000)

In [0]:
PATH_TO_DATA = '/gdrive/My Drive/keyword_extraction'

In [0]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [0]:
data = pd.concat([pd.read_json(file, lines=True, encoding='utf-8') for file in files], axis=0, ignore_index=True, sort=False)

In [0]:
data.shape

(17266, 6)

In [0]:
data.head(1)

,keywords,title,url,abstract,content,summary
0,"[ЭТНОМИГРАЦИОННЫЕ ПРОЦЕССЫ, ДИАСПОРЫ, НОЦ МИОН ПРИ ИГУ, МЕЖРЕГИОНАЛЬНЫЙ, МЕЖДИСИЦИПЛИНАРНЫЕ ИССЛЕДОВАНИЯ]",От исследовательского центра к исследовательской сети: этномиграционные и диаспоральные исследования в НОЦ МИОН при ИГУ,https://cyberleninka.ru/article/n/ot-issledovatelskogo-tsentra-k-issledovatelskoy-seti-etnomigratsionnye-i-diasporalnye-issledovaniya-v-nots-mion-pri-igu,"Данная статья посвящена развитию этномиграционных и диаспоральных исследований в научно-образовательном центре МИОН при ИГУ. Авторы описывают основные этапы формирования междисциплинарного коллектива за последние 20 лет, основные результаты исследовательской работы и ее перспективы.The article examines the development of diaspora, migration and ethnic studies in the Irkutsk CASE functioning under the ISU. The authors consider main periods of formation of interregional and interdisciplinary staff for recent 20 years, main results of the research work by the Irkutsk CASE and its prospects","ЭТНОПОЛИТИЧЕСКИЕ,\nМИГРАЦИОННЫЕ И ДИАСПОРАЛЬНЫЕ ПРОЦЕССЫ\nВ ПЕРЕСЕЛЕНЧЕСКОМ ОБЩЕСТВЕ /\nETHNOPOLITICAL, MIGRATORY, AND DIASPORA'S\nPROCESSES IN THE RESETTLEMENT SOCIETY\nСерия «Политология. Религиоведение»\n2012. № 2 (9), ч. 2. С. 5–12\nОнлайн-доступ к журналу:\nhttp://isu.ru/izvestia\n\nИЗВЕСТИЯ\nИркутского\nгосударственного\nуниверситета\n\nУДК 314.7\n\nОт исследовательского центра\nк исследовательской сети: этномиграционные\nи диаспоральные исследования в НОЦ МИОН при ИГУ*\nК. В. Григоричев, В. И. Дятлов\nНаучно-образовательный центр Межрегиональный институт\nобщественных наук при Иркутском государственном университете,\nИркутский государственный университет, г. Иркутск\nДанная статья посвящена развитию этномиграционных и диаспоральных исследований в научно-образовательном центре МИОН при ИГУ. Авторы описывают основные этапы формирования межрегионального междисциплинарного коллектива за\nпоследние 20 лет, основные результаты исследовательской работы НОЦ МИОН при\nИГУ и ее перспе...",NaN


In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [0]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


### Бейзлайн

In [0]:
tfidf = TfidfVectorizer()

In [0]:
tfidf.fit(data['content'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content'])

In [0]:
texts_vectors.getrow(0).toarray().argsort()[0,:-11:-1]

array([366589, 452726, 109689, 340186, 378920, 474377, 326570, 381182,
       312079, 496467])

In [0]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [0]:
keywords[:3]

[['игу',
  'мион',
  'irkutsk',
  'дятлов',
  'иркутск',
  'научно',
  'диаспоры',
  'исследовательской',
  'григоричев',
  'ноц'],
 ['бурятии',
  'бурятия',
  'на',
  'острогов',
  'республики',
  'налогу',
  'транспортной',
  'xvii',
  'месторождении',
  'монголией'],
 ['10',
  'галактик',
  'lk',
  '11',
  '12',
  'сверхскоплении',
  '13',
  'sdss',
  'пар',
  '2masx']]

In [0]:
evaluate(data['keywords'], keywords)

Precision -  0.05
Recall -  0.08
F1 -  0.06
Jaccard -  0.03


### 1 способ - стемминг, удаление стоп-слов, нормализация

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [stemmer.stem(word) for word in words if word and word not in stops]

    return words

In [0]:
def normalize2(text):
    
    words = [stemmer.stem(word) for word in text if word and word not in stops]

    return words

In [0]:
%%time 
data['content_norm'] = data['content'].apply(normalize)


CPU times: user 14min 59s, sys: 1.35 s, total: 15min
Wall time: 15min 1s


In [0]:
%%time
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.04
Recall -  0.06
F1 -  0.05
Jaccard -  0.02
CPU times: user 4.18 s, sys: 9.99 ms, total: 4.19 s
Wall time: 4.19 s


In [0]:
%%time 
data['keywords_stem'] = data['keywords'].apply(normalize2)

CPU times: user 7.95 s, sys: 19 ms, total: 7.96 s
Wall time: 7.96 s


In [0]:
%%time
evaluate(data['keywords_stem'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.16
F1 -  0.12
Jaccard -  0.07
CPU times: user 3.6 s, sys: 16 ms, total: 3.61 s
Wall time: 3.61 s


### Вывод:
Использование стемминга и удаление стоп-слов как в колонке content, так и в колонке keywords позволило значительно улучшить результаты бейзлайна. Стемминг показал себя хуже, чем лемматизация, но так как я не могу применить последнюю - это неплохой вариант. 

### 2 способ - настройка tfidf-векторизатора

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [stemmer.stem(word) for word in words if word and word not in stops]

    return words

In [0]:
tfidf = TfidfVectorizer(tokenizer=normalize, min_df=10, max_df=0.9) #stop_words=stops,

In [0]:
tfidf.fit(data['content'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=None,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function normalize at 0x7fdff5ade7b8>, use_idf=True,
                vocabulary=None)

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content'])

In [0]:
texts_vectors.getrow(0).toarray().argsort()[0,:-11:-1]

array([19547, 19718, 33835,  3752, 23937, 16903, 16014, 25846, 25840,
       30215])

In [0]:
%%time
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

CPU times: user 20.4 s, sys: 171 ms, total: 20.5 s
Wall time: 20.5 s


In [0]:
evaluate(data['keywords'], keywords)

Precision -  0.04
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


In [0]:
evaluate(data['keywords_stem'], keywords)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.07


### Вывод:
Использование стемминга, удаление стоп-слов и tf-idf векторизация с измененными параметрами дало лучший результат из всех опробованных мною способов. 

### 3 способ - графы с пересекающимися окнами

In [0]:
def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [0]:
def get_kws(text, top=5, random_p=0.1):
    try:
      vocab = set(text)
      m = np.zeros((len(vocab), len(vocab)))
      word2id = {w:i for i, w in enumerate(vocab)}
      id2word = {i:w for i, w in enumerate(vocab)}
      # преобразуем слова в индексы для удобства
      ids = [word2id[word] for word in text]

    # пройдемся окном по всему тексту
      for i in range(0, len(ids)):
          window = ids[i:i+15]
          # добавим единичку всем парам слов в этом окне
          for j, k in combinations(window, 2):
              # чтобы граф был ненаправленный 
              m[j][k] += 1
              m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
      for i in range(m.shape[0]):
          s = np.sum(m[i])
          if not s:
              continue
          m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
      c = Counter()
    # начнем с абсолютного случайно выбранного элемента
      n = np.random.choice(len(vocab))
      for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
          go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
          if go_random:
              n = np.random.choice(len(vocab))
        
        
        ### 
          n = take_step(n, m)
        # записываем узлы, в которых были
          c.update([n])
    
    except ValueError:
      pass
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

In [0]:
def normalize_lite(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word and word not in stops]

    return words

In [0]:
%%time
data['content_norm_lite'] = data['content'].apply(normalize_lite)

CPU times: user 11.5 s, sys: 1.96 s, total: 13.5 s
Wall time: 13.5 s


In [0]:
data['content_norm_lite'].head(5)

0    [этнополитические, миграционные, диаспоральные, процессы, переселенческом, обществе, ethnopolitical, migratory, and, diaspora's, processes, in, the, resettlement, society, серия, политология, религиоведение, 2012, 2, 9, ч, 2, 5–12, онлайн-доступ, журналу, http://isu.ru/izvestia, известия, иркутского, государственного, университета, удк, 314.7, исследовательского, центра, исследовательской, сети, этномиграционные, диаспоральные, исследования, ноц, мион, игу, григоричев, дятлов, научно-образовательный, центр, межрегиональный, институт, общественных, наук, иркутском, государственном, университете, иркутский, государственный, университет, г, иркутск, данная, статья, посвящена, развитию, этномиграционных, диаспоральных, исследований, научно-образовательном, центре, мион, игу, авторы, описывают, основные, этапы, формирования, межрегионального, междисциплинарного, коллектива, последние, 20, лет, основные, результаты, исследовательской, работы, ноц, мион, игу, перспективы, специальная, ча

In [0]:
%%time
for i, line in enumerate(data['keywords']):
  for j, exp in enumerate(line):
    #print(i, j, exp.lower())
    word = data['keywords'][i][j]
    low_word = word.lower()
    data['keywords'][i][j] = low_word

CPU times: user 4.06 s, sys: 0 ns, total: 4.06 s
Wall time: 4.06 s


### Окно из 5 слов

In [0]:
keywords_rw = data['content_norm_lite'].apply(lambda x: get_kws(x))

In [0]:
evaluate(data['keywords'], keywords_rw) 

Precision -  0.06
Recall -  0.04
F1 -  0.05
Jaccard -  0.03


### Окно из 10 слов

In [0]:
keywords_rw = data['content_norm_lite'].apply(lambda x: get_kws(x))

In [0]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


### Окно из 3 слов

In [0]:
keywords_rw = data['content_norm_lite'].apply(lambda x: get_kws(x))

In [0]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.05
Recall -  0.04
F1 -  0.04
Jaccard -  0.02


### Окно из 15 слов

In [0]:
%%time 
keywords_rw = data['content_norm_lite'].apply(lambda x: get_kws(x))

CPU times: user 1h 8min 12s, sys: 15.8 s, total: 1h 8min 28s
Wall time: 1h 8min 28s


In [0]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


### Вывод:
Использование факторов с пересекающимися окнами позволило улучшить результат бейзлайна Precision на 0.1. Остальные метрики немного хуже. Возможно получилось бы значительно улучшить результат, использовав лемматизацию Pymorphy, но на таком количестве данных google colab и jupyter на windows не позволяют этого сделать.